# RAG Quickstart

Set up a simple Question-Answering system with LangChain and CassIO, using Cassandra / Astra DB as the Vector Database.

## Colab-specific setup

Make sure you have a database available. This is a Colab notebook, so here it is assumed you will use
an Astra DB cloud instance. For usage with an Apache Cassandra® cluster, switch to a local setup
as instructed [on cassio.org](https://cassio.org/more_info/#use-a-local-vector-capable-cassandra).

Get ready to supply the connection parameters: Database ID and Token string
(see [Pre-requisites](https://cassio.org/start_here/#vector-database) on cassio.org for details.
Remember you need a Token with role [Database Administrator](https://awesome-astra.github.io/docs/pages/astra/create-token/)).

Likewise, ensure you have the necessary secret(s) for the LLM provider of your choice: you'll be asked to input it shortly
(see [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for details).

_Note: this notebook is part of the CassIO documentation. Visit [this page on cassIO.org](https://cassio.org/frameworks/langchain/qa-basic/)._


In [ ]:
# install required dependencies
! pip install -q --progress-bar off \
    "cassio>=0.1.5" \
    "jupyter>=1.0.0" \
    "langchain>=0.1.16,<0.2" \
    "langchain-openai>=0.1.3,<0.2" \
    "langchain-google-vertexai>=1.16.0,<2" \
    "python-dotenv==1.0.1" 
exit()

⚠️ **Do not mind a "Your session crashed..." message you may see.**

It was us, making sure your kernel restarts with all the correct dependency versions. _You can now proceed with the notebook._

### DB connection parameters
Example values (check your Astra dashboard):
- Database ID example: `01234567-89ab-cdef-0123-456789abcdef`
- Token example: `AstraCS:6gBhNmsk135....` (ensure it has a role of at least "Database Administrator")
- _Keyspace. Optional, string if provided. Example:_ `default_keyspace`

In [ ]:
import os
from getpass import getpass

# Input your Astra DB connection parameters and secrets:

os.environ["ASTRA_DB_ID"] = input("ASTRA_DB_ID = ")

os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("ASTRA_DB_APPLICATION_TOKEN = ")

_keyspace = input("(Optional) ASTRA_DB_KEYSPACE = ")
if _keyspace:
    os.environ["ASTRA_DB_KEYSPACE"] = _keyspace

### LLM Provider

In the cell below you can choose between **GCP Vertex AI** or **OpenAI** for your LLM services.
(See [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for more details).

Make sure you set the `llmProvider` variable and supply the corresponding access secrets in the following cell.

In [ ]:
# Set your secret(s) for LLM access:
llmProvider = 'OpenAI'  # 'GCP_VertexAI', 'Azure_OpenAI'


In [ ]:
from getpass import getpass
if llmProvider == 'OpenAI':
    apiSecret = getpass(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'GCP_VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
elif llmProvider == 'Azure_OpenAI':
    # a few parameters must be input
    apiSecret = input(f'Your API Key for LLM provider "{llmProvider}": ')
    os.environ['AZURE_OPENAI_API_KEY'] = apiSecret
    apiBase = input('The "Base URL" for your models (e.g. "https://YOUR-RESOURCE-NAME.openai.azure.com"): ')
    os.environ['AZURE_OPENAI_ENDPOINT'] = apiBase
    apiLLMDepl = input('The name of your LLM Deployment: ')
    os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'] = apiLLMDepl
    apiLLMModel = input('The name of your LLM Model (e.g. "gpt-4"): ')
    os.environ['AZURE_OPENAI_LLM_MODEL'] = apiLLMModel
    apiEmbDepl = input('The name for your Embeddings Deployment: ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'] = apiEmbDepl
    apiEmbModel = input('The name of your Embedding Model (e.g. "text-embedding-ada-002"): ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_MODEL'] = apiEmbModel

    # The following is probably not going to change for some time...
    os.environ['AZURE_OPENAI_API_VERSION'] = '2023-03-15-preview'
else:
    raise ValueError('Unknown/unsupported LLM Provider')

In [ ]:
# retrieve the text of a few short stories that will be indexed in the vector store
! mkdir texts
! curl https://raw.githubusercontent.com/CassioML/cassio-website/main/docs/frameworks/langchain/texts/amontillado.txt --output texts/amontillado.txt
! curl https://raw.githubusercontent.com/CassioML/cassio-website/main/docs/frameworks/langchain/texts/manuscript.txt --output texts/manuscript.txt
! curl https://raw.githubusercontent.com/CassioML/cassio-website/main/docs/frameworks/langchain/texts/mask.txt --output texts/mask.txt

### Colab preamble completed

The following cells constitute the demo notebook proper.

# RAG Quickstart

Set up a simple Question-Answering system with LangChain and CassIO, using Cassandra / Astra DB as the Vector Database.

_**NOTE:** this uses Cassandra's "Vector Similarity Search" capability.
Make sure you are connecting to a vector-enabled database for this demo._

In [1]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader

The following line imports the Cassandra flavor of a LangChain vector store:

In [2]:
from langchain_community.vectorstores.cassandra import Cassandra

A database connection is needed. _(If on a Colab, the only supported option is the cloud service Astra DB.)_

In [3]:
# Getting ready to initialize the DB connection globally ...
import os

import cassio

Select your choice of database by editing this cell, if needed:

In [4]:
database_mode = "astra_db"  # only "astra_db" supported on Colab

In [5]:
if database_mode == "astra_db":
    cassio.init(
        database_id=os.environ["ASTRA_DB_ID"],
        token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
        keyspace=os.environ.get("ASTRA_DB_KEYSPACE"),  # this is optional
    )

In [6]:
if database_mode == "cassandra":
    # Cassandra not supported on Colab - please define your own getCassandraCQLSession/getCassandraCQLKeyspace
    cassio.init(
        session=getCassandraCQLSession(),
        keyspace=getCassandraCQLKeyspace(),
    )

Both an LLM and an embedding function are required.

Below is the logic to instantiate the LLM and embeddings of choice. We chose to leave it in the notebooks for clarity.

In [7]:
import os
# creation of the LLM resources

# 'GCP_VertexAI', 'OpenAI' or 'Azure_OpenAI'

if llmProvider == 'OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'open_ai'
    from langchain_openai import OpenAIEmbeddings
    from langchain_openai import OpenAI

    llm = OpenAI(temperature=0)
    myEmbedding = OpenAIEmbeddings()

    print('LLM+embeddings from OpenAI')
elif llmProvider == 'GCP_VertexAI':
    from langchain_google_vertexai import VertexAI
    from langchain_google_vertexai import VertexAIEmbeddings
    
    llm = VertexAI(model_name="text-bison")
    myEmbedding = VertexAIEmbeddings(model_name="textembedding-gecko@003")

    print('LLM+embeddings from Vertex AI')
elif llmProvider == 'Azure_OpenAI':
    from langchain_openai import AzureChatOpenAI
    from langchain_openai import AzureOpenAIEmbeddings

    myEmbedding = AzureOpenAIEmbeddings(
        #model=os.environ['AZURE_OPENAI_EMBEDDINGS_MODEL'],    
        azure_deployment=os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'],
        openai_api_version=os.environ['AZURE_OPENAI_API_VERSION'],
    )
    llm = AzureChatOpenAI(
        openai_api_version=os.environ['AZURE_OPENAI_API_VERSION'],
        azure_deployment=os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'],
    )
    
    print('LLM+embeddings from Azure OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

LLM+embeddings from Vertex AI


## A minimal example

The following is a minimal usage of the Cassandra vector store. The store is created and filled at once, and is then queried to retrieve relevant parts of the indexed text, which are then stuffed into a prompt finally used to answer a question.

The following creates an "index creator", which knows about the type of vector store, the embedding to use and how to preprocess the input text:

_(Note: stores built with different embedding functions will need different tables. This is why we append the `llmProvider` name to the table name in the next cell.)_

In [8]:
table_name = 'vs_test1_' + llmProvider

index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Cassandra,
    embedding=myEmbedding,
    text_splitter=CharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=0,
    ),
    vectorstore_kwargs={
        'session': None,  # i.e. fall back to cassio.init setting
        'keyspace': None,  # i.e. fall back to cassio.init setting
        'table_name': table_name,
    },
)

Loading a local text (a short story by E. A. Poe will do)

In [9]:
loader = TextLoader('texts/amontillado.txt', encoding='utf8')

This takes a few seconds to run, as it must calculate embedding vectors for a number of chunks of the input text:

In [10]:
index = index_creator.from_loaders([loader])

### Check what's on DB

By way of demonstration, if you were to directly read the rows stored in your database table, this is what you would now find there (not that you'll ever _have to_, for LangChain and CassIO provide an abstraction on top of that):

In [11]:
c_session = cassio.config.resolve_session()
c_keyspace = cassio.config.resolve_keyspace()


cqlSelect = f'SELECT * FROM {c_keyspace}.{table_name} LIMIT 2;'
# This query is OK for casual inspection, not very good in prod as is
rows = c_session.execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    print(f'    row_id:            {row.row_id}')
    print(f'    vector:            {str(row.vector)[:64]} ...')
    print(f'    body_blob:         {row.body_blob[:64]} ...')
    print(f'    metadata_s:        {row.metadata_s}')        

print('\n...')


Row 0:
    row_id:            dd62fa9809554baca7dce78be69fe426
    vector:            [0.004388920962810516, -0.008436674252152443, -0.067666538059711 ...
    body_blob:         "Nitre," I replied.  "How long have you had that cough?"

"Ugh!  ...
    metadata_s:        {'source': 'texts/amontillado.txt'}

Row 1:
    row_id:            2aa28a2d16244a36b661898c55100101
    vector:            [0.05901625007390976, 0.001555337687022984, -0.0765535980463028, ...
    body_blob:         There were no attendants at home; they had absconded to make mer ...
    metadata_s:        {'source': 'texts/amontillado.txt'}

...


### Ask a question, get an answer

In [12]:
query = "Who is Luchesi?"
index.query(query, llm=llm)

" The provided text does not give a clear answer to who Luchesi is. It only mentions that Luchesi cannot tell the difference between Amontillado and Sherry, and that the narrator's friend Fortunato believes Luchesi's taste in wine is comparable to the narrator's own."

## Spawning a "retriever" from the index

You just saw how easily you can plug a Cassandra-backed Vector Index into a full question-answering LangChain pipeline.

But you can as easily work at a slightly lower level: the following code spawns a `VectorStoreRetriever` from the index for manual [retrieval](https://python.langchain.com/en/latest/modules/indexes/retrievers.html) of documents related to a given query text. The results are instances of LangChain's `Document` class.

In [13]:
retriever = index.vectorstore.as_retriever(search_kwargs={
    'k': 2,
})

In [14]:
retriever.get_relevant_documents(
    "Check the motto of the Montresors"
)

[Document(page_content='"A huge human foot d\'or, in a field azure; the foot crushes a serpent\nrampant whose fangs are imbedded in the heel."\n\n"And the motto?"\n\n"_Nemo me impune lacessit_."\n\n"Good!" he said.', metadata={'source': 'texts/amontillado.txt'}),
 Document(page_content='He raised it to his lips with a leer.  He paused and nodded to me\nfamiliarly, while his bells jingled.\n\n"I drink," he said, "to the buried that repose around us."\n\n"And I to your long life."\n\nHe again took my arm, and we proceeded.\n\n"These vaults," he said, "are extensive."\n\n"The Montresors," I replied, "were a great and numerous family."\n\n"I forget your arms."', metadata={'source': 'texts/amontillado.txt'})]

## What now?

This demo is hosted [here](https://cassio.org/frameworks/langchain/qa-basic/) at cassio.org.

Discover the other ways you can integrate 
Cassandra/Astra DB with your ML/GenAI needs,
right **within [your favorite framework](https://cassio.org/frameworks/langchain/about/)**.